In [1]:
import numpy as np

## Penguins

In [2]:
anfispy_acc = np.load('data/anfispy_acc_penguins.npy')
anfispy_time = np.load('data/anfispy_time_penguins.npy')
anfispy_f1 = np.load('data/anfispy_f1_penguins.npy')
anfispy_inference = np.load('data/anfispy_inference_penguins.npy')

skanfis_hybrid_acc = np.load('data/skanfis_hybrid_acc_penguins.npy')
skanfis_hybrid_time = np.load('data/skanfis_hybrid_time_penguins.npy')
skanfis_hybrid_f1 = np.load('data/skanfis_hybrid_f1_penguins.npy')
skanfis_hybrid_inference = np.load('data/skanfis_hybrid_inference_penguins.npy')

skanfis_online_acc = np.load('data/skanfis_online_acc_penguins.npy')
skanfis_online_time = np.load('data/skanfis_online_time_penguins.npy')
skanfis_online_f1 = np.load('data/skanfis_online_f1_penguins.npy')
skanfis_online_inference = np.load('data/skanfis_online_inference_penguins.npy')

In [3]:
print("\n--- ANFISpy ---")
print(f"Training Time:  {anfispy_time.mean():.4f} ± {anfispy_time.std():.4f} s")
print(f"Inference Time: {anfispy_inference.mean():.4f} ± {anfispy_inference.std():.4f} s")
print(f"Acc:            {anfispy_acc.mean():.4f} ± {anfispy_acc.std():.4f}")
print(f"F1:             {anfispy_f1.mean():.4f} ± {anfispy_f1.std():.4f}")


--- ANFISpy ---
Training Time:  2.4657 ± 0.1195 s
Inference Time: 0.0006 ± 0.0000 s
Acc:            0.9759 ± 0.0181
F1:             0.9754 ± 0.0183


In [4]:
print("\n--- SkANFIS (Hybrid) ---")
print(f"Training Time:  {skanfis_hybrid_time.mean():.4f} ± {skanfis_hybrid_time.std():.4f} s")
print(f"Inference Time: {skanfis_hybrid_inference.mean():.4f} ± {skanfis_hybrid_inference.std():.4f} s")
print(f"Acc:            {skanfis_hybrid_acc.mean():.4f} ± {skanfis_hybrid_acc.std():.4f}")
print(f"F1:             {skanfis_hybrid_f1.mean():.4f} ± {skanfis_hybrid_f1.std():.4f}")


--- SkANFIS (Hybrid) ---
Training Time:  2.5239 ± 0.1391 s
Inference Time: 0.0112 ± 0.0131 s
Acc:            0.9398 ± 0.0353
F1:             0.9398 ± 0.0384


In [5]:
print("\n--- SkANFIS (Online) ---")
print(f"Training Time:  {skanfis_online_time.mean():.4f} ± {skanfis_online_time.std():.4f} s")
print(f"Inference Time: {skanfis_online_inference.mean():.4f} ± {skanfis_online_inference.std():.4f} s")
print(f"Acc:            {skanfis_online_acc.mean():.4f} ± {skanfis_online_acc.std():.4f}")
print(f"F1:             {skanfis_online_f1.mean():.4f} ± {skanfis_online_f1.std():.4f}")


--- SkANFIS (Online) ---
Training Time:  1.6615 ± 0.9544 s
Inference Time: 0.0042 ± 0.0005 s
Acc:            0.9731 ± 0.0207
F1:             0.9729 ± 0.0206


## Sinc

In [6]:
anfispy_rmse = np.load('data/anfispy_rmse_sinc.npy')
anfispy_time = np.load('data/anfispy_time_sinc.npy')
anfispy_mae = np.load('data/anfispy_mae_sinc.npy')
anfispy_inference = np.load('data/anfispy_inference_sinc.npy')

skanfis_hybrid_rmse = np.load('data/skanfis_rmse_sinc.npy')
skanfis_hybrid_time = np.load('data/skanfis_time_sinc.npy')
skanfis_hybrid_mae = np.load('data/skanfis_mae_sinc.npy')
skanfis_hybrid_inference = np.load('data/skanfis_inference_sinc.npy')

skanfis_online_rmse = np.load('data/skanfis_online_rmse_sinc.npy')
skanfis_online_time = np.load('data/skanfis_online_time_sinc.npy')
skanfis_online_mae = np.load('data/skanfis_online_mae_sinc.npy')
skanfis_online_inference = np.load('data/skanfis_online_inference_sinc.npy')

In [7]:
print("\n--- ANFISpy ---")
print(f"Training Time:  {anfispy_time.mean():.4f} ± {anfispy_time.std():.4f} s")
print(f"Inference Time: {anfispy_inference.mean():.4f} ± {anfispy_inference.std():.4f} s")
print(f"RMSE:           {anfispy_rmse.mean():.4f} ± {anfispy_rmse.std():.4f}")
print(f"MAE:            {anfispy_mae.mean():.4f} ± {anfispy_mae.std():.4f}")


--- ANFISpy ---
Training Time:  86.3953 ± 0.4343 s
Inference Time: 0.0007 ± 0.0000 s
RMSE:           0.0427 ± 0.0201
MAE:            0.0271 ± 0.0148


In [8]:
print("\n--- SkANFIS (Hybrid) ---")
print(f"Training Time:  {skanfis_hybrid_time.mean():.4f} ± {skanfis_hybrid_time.std():.4f} s")
print(f"Inference Time: {skanfis_hybrid_inference.mean():.4f} ± {skanfis_hybrid_inference.std():.4f} s")
print(f"RMSE:           {skanfis_hybrid_rmse.mean():.4f} ± {skanfis_hybrid_rmse.std():.4f}")
print(f"MAE:            {skanfis_hybrid_mae.mean():.4f} ± {skanfis_hybrid_mae.std():.4f}")


--- SkANFIS (Hybrid) ---
Training Time:  185.7120 ± 13.2870 s
Inference Time: 0.0084 ± 0.0115 s
RMSE:           0.2094 ± 0.0102
MAE:            0.1697 ± 0.0033


In [9]:
print("\n--- SkANFIS (Online) ---")
print(f"Training Time:  {skanfis_online_time.mean():.4f} ± {skanfis_online_time.std():.4f} s")
print(f"Inference Time: {skanfis_online_inference.mean():.4f} ± {skanfis_online_inference.std():.4f} s")
print(f"RMSE:           {skanfis_online_rmse.mean():.4f} ± {skanfis_online_rmse.std():.4f}")
print(f"MAE:            {skanfis_online_mae.mean():.4f} ± {skanfis_online_mae.std():.4f}")


--- SkANFIS (Online) ---
Training Time:  2.1568 ± 1.7950 s
Inference Time: 0.0045 ± 0.0021 s
RMSE:           0.1099 ± 0.0043
MAE:            0.0746 ± 0.0063
